In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [8]:
ls data

men_shoes.csv  shoes_prices_man.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [12]:
run_model([ 'brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [13]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model([ 'brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [14]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [16]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
#zamiana na małe litery
run_model([ 'brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model([ 'brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [19]:
# Dane  słownikowo
test = {'key': 'value'}
test['key']
str(test)

"{'key': 'value'}"

In [0]:
def parse_features(x):
  output_dict ={}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [46]:
# unikalne wartości w słowniku

keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [50]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [51]:
df.columns


Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_age start', 'feat_hood', 'feat_free item', 'feat_dimensions',
       'feat_ring size', 'feat_chain included', 'feat_insulation',
       'feat_fit to', 'feat_is polarized', 'feat_handbag style'],
      dtype='object', length=526)

In [0]:
keys_stat ={}
for key in keys:
  keys_stat[key] = df[False ==  df[get_name_feat(key)].isnull()].shape[0] /df.shape[0] *100

In [57]:
{k:v for k,v in keys_stat.items() if v > 30} 

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+ '_cat'] = df[get_name_feat(key)].factorize()[0]

In [63]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand][['brand', 'feat_brand']].shape

(8846, 2)

In [87]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_fabrication',
 'feat_catalog',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_clothing category',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_light transmission (vlt)_cat',
 'feat_euro size_cat',
 'feat_isbn_cat',
 'feat_bezel_cat',
 'feat_item style_cat',
 'feat_size_cat',
 'feat_bridge/temple size:_cat',
 'feat_hooded_cat',
 'feat_recommended location_cat',
 'feat_fabric care instructions_cat',
 'feat_материал_cat',
 'feat_reinforced knee_cat',
 'feat_gold karat_cat',
 'feat_batteries included?_cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_motion control_cat',
 'feat_country of origin - components_cat',
 'feat_has mercury_cat',
 'feat_use_cat',
 'feat_to fit_cat',
 'feat_fits brands

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat',  'feat_material_cat', 'feat_style_cat', 'feat_sport_cat', 'feat_metal type_cat', 'feat_shape_cat']
#feats += ['feat_euro size_cat',
# 'feat_isbn_cat',
# 'feat_bezel_cat',
 #'feat_item style_cat',
# 'feat_size_cat',
 #'feat_bridge/temple size:_cat',
 #'feat_hooded_cat',
# 'feat_recommended location_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
results = run_model( feats, model  )

In [94]:
X = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(results)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.261715157995454, 4.2314321918195725)


Weight,Feature
0.2566 ± 0.0077,brand_cat
0.1039 ± 0.0163,feat_material_cat
0.0241 ± 0.0038,feat_gender_cat
0.0180 ± 0.0009,feat_brand_cat
0.0128 ± 0.0002,feat_shape_cat
0.0085 ± 0.0010,feat_metal type_cat
0.0031 ± 0.0014,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [80]:
# wygląda,  zę brand jest ważny wiec je poanalizujmy
df [ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'air force 1', 'pattern': 'solid', 'material': 'synthetic', 'condition': 'new with box'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'sport': 'lacrosse, football', 'country/region of manufacture': 'china', 'type': 'cleats', 'condition': 'new without box'},
       {'gender': 'boys', 'shoe category': 'boys shoes', 'color': 'volt/photo blue/hyper pink', 'casual & dress shoe style': 'athletic', 'manufacturer part number': '644534700', 'brand': 'nike', 'shoe width': 'medium'}],
      dtype=object)

In [96]:
!ls



data  HelloGithub.ipynb  LICENSE  matrix_1  README.md


In [98]:
!git add matrix_1/day5.ipynb



fatal: pathspec 'matrix_1/day5.ipynb' did not match any files
